#Install Large Files

Connect your Google Drive first! Press button on left side.

In [1]:
# Root directory of the project
ROOT_DIR = '/content/drive/My Drive/lego-cnn-master'

In [ ]:
!pip install gdown
import gdown

# Get example model weight
url = 'https://drive.google.com/uc?id=1XjDlMButvwsZsYauUGybhY5aCBA1tBjo'
output = ROOT_DIR + '/snapshots/lego20200717T1532/mask_rcnn_lego_0041.h5'
gdown.download(url, output, quiet=False)

# Get dataset 6
url = 'https://drive.google.com/uc?id=1v4gsJ-2B3LQYKul6XNMrYCbZfIW0M1Wb'
output = ROOT_DIR + '/datasets/images6.zip'
gdown.download(url, output, quiet=False)

# Get dateset 22
url = 'https://drive.google.com/uc?id=1ZmavdrcbDyLNiySJlFPSSpz5vABg1vod'
output = ROOT_DIR + '/datasets/images22.zip'
gdown.download(url, output, quiet=False)

In [4]:
# Unzip the dataset that you want
unzip_dir = ROOT_DIR.replace(" ","\ ")

!unzip -q $unzip_dir/datasets/images22.zip -d $unzip_dir/datasets
!mv $unzip_dir/datasets/images22/* $unzip_dir/datasets/lego/

# Install Packages

In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
import os
import sys

# Prepare colab environment and load images
os.system('pip install keras==2.2.4') # Must be 2.2.4 (or 2.3.0) otherwise get metrics_tensors error
os.system('python setup.py build_ext --inplace; pip install .')
os.chdir(ROOT_DIR)

# Import Packages

In [7]:
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.lego import lego

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


# Infer RPN

### Load Configuration

In [ ]:
LEGO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "snapshots", "lego20200717T1532","mask_rcnn_lego_0041.h5") # Comment out to use snapshot from latest training

DATASET = "eval"
config = lego.LegoConfig()
LEGO_DIR = os.path.join(ROOT_DIR, "datasets", "lego")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceConfig()
config.display()

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

### Load Dataset

In [ ]:
dataset = lego.LegoDataset()
dataset.load_lego(LEGO_DIR, DATASET)
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

### Load Model and Weights

In [ ]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
print("Loading weights ", LEGO_WEIGHTS_PATH)
model.load_weights(LEGO_WEIGHTS_PATH, by_name=True)

### RPN Analysis

In [ ]:
if DATASET == "eval":
    image_id = dataset.get_image_id("0000000002.png") # If eval set us the image with the Lego haufen
else:
    image_ids = np.random.choice(dataset.image_ids, 1)
    image_id = image_ids[0]
    
image, image_meta, gt_class_ids, gt_bboxes, gt_masks = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

In [ ]:
# Run RPN sub-graph
if config.USE_RPN_ROIS:
    pillar = model.keras_model.get_layer("ROI").output

    # TF 1.4 and 1.9 introduce new versions of NMS. Search for all names to support TF 1.3~1.10
    nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression:0")
    if nms_node is None:
        nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV2:0")
    if nms_node is None: #TF 1.9-1.10
        nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV3:0")

    rpn = model.run_graph([image], [
        ("rpn_class", model.keras_model.get_layer("rpn_class").output),         # Last layer in RPN with the proposed class probabilites for background or foreground per anchor
        ("pre_nms_anchors", model.ancestor(pillar, "ROI/pre_nms_anchors:0")),
        ("refined_anchors", model.ancestor(pillar, "ROI/refined_anchors:0")),
        ("refined_anchors_clipped", model.ancestor(pillar, "ROI/refined_anchors_clipped:0")),
        ("post_nms_anchor_ix", nms_node),                                       # Filtered anchors that have a NMS of RPN_NMS_THRESHOLD (>0.7)
        ("proposals", model.keras_model.get_layer("ROI").output),               # "ROI" is the name of the ProposalLayers, the filtered rois proposals
    ], config)

In [ ]:
# Find the proposal that has the highest IoU on each GT box and take average of all
max_proposals_bbox = np.empty((gt_bboxes.shape[0], 4))
max_proposals_iou = np.empty((gt_bboxes.shape[0], 1))

# Get proposals from graph
h, w = config.IMAGE_SHAPE[:2]
proposals = rpn['proposals'][0, :] * np.array([h, w, h, w])

print("Best proposals are:\n")
for i, box in enumerate(gt_bboxes):

    overlaps = utils.compute_overlaps(np.array([box.tolist()]), proposals)
    iou_max = np.max(overlaps, axis=1)
    iou_argmax = np.argmax(overlaps, axis=1)
    max_proposals_bbox[i] = proposals[iou_argmax]
    max_proposals_iou[i] = iou_max

    y1, x1, y2, x2 = proposals[iou_argmax][0]
    iou = iou_max[0]

    print("Proposal Bbox {}: \t[{:.0f} {:.0f} {:.0f} {:.0f}] w={:.0f} h={:.0f} iou={:.2f}".format(i, x1, y1, x2, y2, x2-x1, y2-y1, iou))


print("\nAverage maximum IoU is: {0:0.2f}".format(np.average(max_proposals_iou)))
visualize.draw_boxes(image, boxes=gt_bboxes, refined_boxes=max_proposals_bbox)

# Run Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
APs = []
TPs = []
FPs = []
total_instances = 0

# IoU threshold to determine a postive match
mAP_IOU_THRESHOLD = 0.5

if DATASET == "eval":
    image_ids = dataset.image_ids
else:
    image_ids = np.random.choice(dataset.image_ids, 5 if config.USE_STAGE_TWO else 1)

for image_id in image_ids:

    # Load image and ground truth data
    image, image_meta, gt_class_ids, gt_bboxes, gt_masks =  modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

    # Run object detection
    results = model.detect([image], gt_class_ids, gt_bboxes, gt_masks, verbose=0)
    r = results[0]

    ax = get_ax(1)
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset.class_names, gt_bboxes, gt_class_ids, 
                                r['scores'], ax=ax, title="Predictions")

    # Compute AP, IoU ist mit den Masken berechnet, nicht mit Bboxen
    AP, precisions, recalls, overlaps, true_positives, false_positives = utils.compute_ap(  gt_bboxes, gt_class_ids, gt_masks,    
                                                                                            r["rois"], r["class_ids"], r["scores"], r['masks'],
                                                                                            iou_threshold=mAP_IOU_THRESHOLD)
    if true_positives == 0 and false_positives == 0:
        print("\n*** No instances matched in {} *** \n".format(dataset.image_info[image_id]["id"]))
        
    APs.append(AP)
    TPs.append(true_positives)
    FPs.append(false_positives)
    total_instances += len(gt_bboxes)

tps = int(np.sum(TPs))
fps = int(np.sum(FPs))

print('TP: {:2.0f}   FP: {:2.0f}   TP/FP: {:2.1f}   TP/Total: {:2.0f}%'.format(tps, fps, 0 if fps == 0 else (tps/fps), 0 if total_instances == 0 else (100.0/total_instances)*tps))
print("mAP@" + str(mAP_IOU_THRESHOLD)+ ": ", np.mean(APs))


# Train CNN

In [ ]:
import samples.lego.lego
import mrcnn.model
import mrcnn.utils
import mrcnn.config
import imp

# If you change code on the fly, reload modules
import imp
imp.reload(mrcnn.config)
imp.reload(samples.lego.lego)
imp.reload(mrcnn.model)
imp.reload(mrcnn.utils)

from samples.lego.lego import main as tmain
from google.cloud import storage

# Configuration section
PATH_DATASET                = os.path.join('datasets', 'lego')
USE_PREV_WEIGHTS			      = False # Set to True if you want to restart from a previous session
NB_OF_EPOCHS                = 1 # Typically 40 epoches are sufficient for this project, 

# Where are we
print('\Script running in: ' + os.getcwd())
print("Python version: " + sys.version)
print("Python install path: " + sys.executable)

# Show tool version
import tensorflow
print(tensorflow.__version__)
import keras
print(keras.__version__)


# Call train function
if USE_PREV_WEIGHTS:
    LEGO_WEIGHTS_PATH = tmain([ 'train', '--dataset=' + PATH_DATASET, '--logs=snapshots', '--enable-augmentation', '--weights=last','--epochs=' + str(NB_OF_EPOCHS)])
else:
    LEGO_WEIGHTS_PATH = tmain([ 'train', '--dataset=' + PATH_DATASET, '--logs=snapshots','--enable-augmentation', '--epochs=' + str(NB_OF_EPOCHS)])
